In [0]:
MERGE INTO workspace.growth_poc.silver_residents AS target
USING (
  SELECT
    l.Nationality,
    l.English_Nationality,
    l.ISO_Code,
    l.Primary_Language,
    l.Language_Code,
    'long-term' AS Resident_Category,
    l.Year,
    l.Amount
  FROM
    workspace.growth_poc.silver_residents_longterm l
  WHERE l.TimeStamp = (SELECT MAX(TimeStamp) FROM workspace.growth_poc.silver_residents_longterm)
  UNION ALL
  SELECT
    s.Nationality,
    s.English_Nationality,
    s.ISO_Code,
    s.Primary_Language,
    s.Language_Code,
    'short-term' AS Resident_Category,
    s.Year,
    s.Amount
  FROM
    workspace.growth_poc.silver_residents_shortterm s
  WHERE s.TimeStamp = (SELECT MAX(TimeStamp) FROM workspace.growth_poc.silver_residents_shortterm)
) AS source
-- defining joining keys
ON TRIM(target.Nationality) = TRIM(source.Nationality)
   AND target.Year = source.Year
   AND target.Resident_Category = source.Resident_Category

WHEN MATCHED THEN
  UPDATE SET
    target.Nationality_English = source.English_Nationality,
    target.ISO_Code = source.ISO_Code,
    target.Primary_Language = source.Primary_Language,
    target.Language_Code = source.Language_Code,
    target.Amount = source.Amount,
    target.TimeStamp = current_timestamp()  

WHEN NOT MATCHED THEN
  INSERT (
    Nationality,
    Nationality_English,
    ISO_Code,
    Primary_Language,
    Language_Code,
    Resident_Category,
    Year,
    Amount,
    TimeStamp
  )
  VALUES (
    source.Nationality,
    source.English_Nationality,
    source.ISO_Code,
    source.Primary_Language,
    source.Language_Code,
    source.Resident_Category,
    source.Year,
    source.Amount,
    current_timestamp()  
  );

In [0]:
-- CREATE TABLE workspace.growth_poc.silver_residents AS
-- SELECT
--     l.Nationality,
--     l.English_Nationality AS Nationality_English,
--     l.ISO_Code,
--     l.Primary_Language,
--     l.Language_Code,
--     'long-term' AS Resident_Category,
--     l.Year,
--     l.Amount,
--     getdate() AS TimeStamp

-- FROM
--     workspace.growth_poc.silver_residents_longterm l
-- WHERE l.TimeStamp = (SELECT MAX(TimeStamp) FROM workspace.growth_poc.silver_residents_longterm)
-- UNION ALL
-- SELECT
--     s.Nationality,
--     s.English_Nationality AS Nationality_English,
--     s.ISO_Code,
--     s.Primary_Language,
--     s.Language_Code,
--     'short-term' AS Resident_Category,
--     s.Year,
--     s.Amount,
--     getdate() AS TimeStamp
-- FROM
--     workspace.growth_poc.silver_residents_shortterm s
-- WHERE s.TimeStamp = (SELECT MAX(TimeStamp) FROM workspace.growth_poc.silver_residents_shortterm)
 